# Verificación de la simulación del filtro FIR
## Condición $K=M, n_0=0, k_0=0$  y DFT-FB

### Diseño
Este script proviene de check_fir.py. Compara la salida de las simulaciones con Python y la que obtengo de simular el FIR con Vivado.

El proyecto correspondiente está en **/home/arnaldi/vtests/fir_test**

Importo los modulos necesarios

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import lfilter, kaiser
from scipy.fftpack import fft, fftfreq, fftshift
from dsp_funcs import h_freqz, mfreqz

Configuración.

In [ ]:
Fs            = 125e6    # Sampling frequency
M             = 16       # downsampling ratio
I             = 1        # oversampling factor
K             = M*I      # number of channels
k0            = 0        # 
n0            = 0
data_len       = 2**16
prototipo      = '../../coefficients/coeff/prototipo.csv';
c_frac_width  = 15       # fractional whdth for data
write_golden_data = 1;

Primero todo lo relacionado con los archivos de antena, que salen del transmisor (en python) y que uso como entrada en las simulaciones de Vivado.

Esto es ant_source_{real/imag}.dat. La finalización en _ckecks_ es para comprobar el efecto del truncamiento en los datos.

In [ ]:
ant_coe_r       = '../../coefficients/coeff/ant_source_real_cosine_16_checks.coe'
ant_coe_i       = '../../coefficients/coeff/ant_source_imag_cosine_16_checks.coe'

Cargo la señal de entrada. Esto es para comprobar que lo que estoy leyendo está bien. Lo que sigue es la serie de conversiones que debo hacer para tener los datos nuevamente similares a como salen en  ant_source_{real/imag}.dat, nada más que ahora están como coeficientes con sus respectivas conversiones a datos truncados al número necesario de bits que voy a usar en la realidad.

In [ ]:
with open(ant_coe_r, 'r') as fp:
    buff=fp.read()
#convierto a enteros de 16 bits
din_coe_r= np.array([int(s,16) for s in buff.split(',')])

with open(ant_coe_i, 'r') as fp:
    buff=fp.read()
#convierto a enteros de 16 bits
din_coe_i= np.array([int(s,16) for s in buff.split(',')])

a=np.array([din_coe_r[i]/2**c_frac_width if din_coe_r[i] <= 2**c_frac_width else
    (din_coe_r[i]/2**c_frac_width)-2 for i in range(din_coe_r.size)])
b=np.array([din_coe_i[i]/2**c_frac_width if din_coe_i[i] <= 2**c_frac_width else
    (din_coe_i[i]/2**c_frac_width)-2 for i in range(din_coe_i.size)])

din_coe_tot = a + 1j*b

Esto es lo mismo que ant_{real/imag}_cosine_data.dat en gen_tonos_mkids.py

In [ ]:
coe_datas = np.reshape(din_coe_tot,(K,int(din_coe_tot.size/K)), order='F')

Grafico el espectro de la señal completa fdm después de rearmarla.

In [ ]:
#plt.figure()
print('Plot combined spectrum....');
fig,ax=plt.subplots(figsize=(10,6))
#NFFT=din_coe_tot.size
NFFT=1024
f=fftfreq(NFFT,d=1/Fs)
#ww=kaiser(K*data_len,8);
ww = kaiser(NFFT, beta=8)
#max_val   = max(abs(fft(din_coe_tot*ww,NFFT)));
max_vali  = max(abs(fft(din_coe_tot,NFFT)));
ax.plot(fftshift(f),20*np.log10((abs(fftshift(fft(din_coe_tot,NFFT))))/max_vali),'b');
#ax.plot(fftshift(f),20*np.log10((abs(fftshift(fft(din_coe_tot*ww,NFFT))))/max_val),'b');
#plt.plot(f,20*np.log10((abs(fft(ant_ideal,NFFT)))/max_val),'b');
for i in range(int(K/2)):
    ax.vlines((Fs/K)*i,-120,0,ls='dotted',color='red')
for i in reversed(range(int(K/2))):
    ax.vlines(-(Fs/K)*i,-120,0,ls='dotted',color='red')

#ax.vlines(-(Fs/K)*15+156.0e3,-150,0,color='green')
#ax.vlines(-(Fs/K)*15-180.0e3,-150,0,color='orange')
#plt.plot(f,20*np.log10(abs(fft(ant_ideal))/max_val),'b');
#ax.set_title('Espectro combinado del transmisor');#Transmitter combined spectrum');
#plt.legend('Ideal');
#axis tight
ax.axis([-Fs/2, Fs/2, -60, 2])
ax.set_ylabel('Amplitud (dB)')
ax.set_xlabel('Frecuencia (Hz)')

plt.show()

Datos que uso en la simulacion de Python

In [ ]:
#fname_ideal_in_r = '../../golden_data/tx_source_real_cosine.dat'
#fname_ideal_in_i = '../../golden_data/tx_source_imag_cosine.dat'

# Salida de Vivado

Datos que obtengo luego de simular el FIR en Vivado.

In [ ]:
viv_fir_op_r = '/home/arnaldi/work/vtests/fir_test/fir_test.sim/sim_1/behav/xsim/fir_op_real_cosine.dat';
viv_fir_op_i = '/home/arnaldi/work/vtests/fir_test/fir_test.sim/sim_1/behav/xsim/fir_op_imag_cosine.dat';
#fir_op_1_r = '../axis_rxchan16.sim/sim_1/behav/xsim/fir1_op_real_cosine.dat';
#fir_op_1_i = '../axis_rxchan16.sim/sim_1/behav/xsim/fir1_op_imag_cosine.dat';
#fir_op_0_r = 'data_ant/fir0_op_real_cosine.dat';
#fir_op_0_i = 'data_ant/fir0_op_imag_cosine.dat';
#fir_op_1_r = 'data_ant/fir1_op_real_cosine.dat';
#fir_op_1_i = 'data_ant/fir1_op_imag_cosine.dat';


viv_r_fir_op=np.loadtxt(viv_fir_op_r); # senial I(n)
viv_i_fir_op=np.loadtxt(viv_fir_op_i); # senial  Q(n)
#i_fir1_sim=np.loadtxt(fir_op_1_r); # senial I(n)
#q_fir1_sim=np.loadtxt(fir_op_1_i); # senial  Q(n)
viv_x_fir_op=viv_r_fir_op + 1j*viv_i_fir_op #armamos la senial compleja
#x_fir1_sim=i_fir1_sim + 1j*q_fir1_sim; #armamos la senial compleja

Tomamos el número correcto de muestras.

In [ ]:
viv_r_fir_op=np.resize(viv_r_fir_op,viv_r_fir_op.size - len(viv_r_fir_op)%K)
viv_i_fir_op=np.resize(viv_i_fir_op,viv_i_fir_op.size - len(viv_i_fir_op)%K)
viv_x_fir_op=np.resize(viv_x_fir_op,viv_x_fir_op.size - len(viv_x_fir_op)%K)
#i_fir1_sim=np.resize(i_fir1_sim,i_fir1_sim.size - len(i_fir1_sim)%K)
#q_fir1_sim=np.resize(q_fir1_sim,q_fir1_sim.size - len(q_fir1_sim)%K)
#x_fir1_sim=np.resize(x_fir1_sim,x_fir1_sim.size - len(x_fir1_sim)%K)

Datos ideales (Python)

In [ ]:
ideal_op_r    = '../../golden_data/rx_fir_op_ideal_real_cosine.dat';
ideal_op_i    = '../../golden_data/rx_fir_op_ideal_imag_cosine.dat';

i_ideal=np.loadtxt(ideal_op_r); # senial I(n)
q_ideal=np.loadtxt(ideal_op_i); # senial  Q(n)
x_ideal=i_ideal + 1j*q_ideal; #armamos la senial compleja

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
#fig.set_size_inches((10,6))
#fig.suptitle('Input Spectrum',fontsize=18)

#NFFT=viv_x_fir_op.size
NFFT=1024
ww_viv = kaiser(viv_x_fir_op.size, beta=8)
ww_ideal = kaiser(x_ideal.size, beta=8)
X_viv=fft(viv_x_fir_op*ww_viv,NFFT)
X_ideal=fft(x_ideal*ww_ideal,NFFT)
freq=fftfreq(NFFT,d=1/Fs)
#for n in range(K):
ax.plot(fftshift(freq),20*np.log10(fftshift(abs(X_ideal/max(abs(X_ideal))))),'r',label='Python')
ax.plot(fftshift(freq),20*np.log10(fftshift(abs(X_viv/max(abs(X_viv))))),label='Vivado')
#ax.plot(fftshift(freq),20*np.log10(fftshift(abs(X_viv))),label='Vivado')
#ax.plot(fftshift(freq),20*np.log10(fftshift(abs(X_ideal))),label='Python')

for i in range(int(K/2)):
    ax.vlines((Fs/K)*i,-120,0,linestyle='dotted',color='red')
for i in reversed(range(int(K/2))):
    ax.vlines(-(Fs/K)*i,-120,0,linestyle='dotted',color='red')

ax.set_xlim(-Fs/2,Fs/2)
ax.set_ylim(-55,2)
ax.grid(ls='dotted')
ax.legend(loc=0,fontsize=12)
ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
ax.set_ylabel(r'Amplitud (dB)',fontsize=14)
ax.set_xlabel('Frecuencia (Hz)',fontsize=14)
plt.tight_layout()
plt.savefig('plots/fir_out_viv_py.pdf')
plt.show()

Recuperar el filtro prototipo 

In [ ]:
proto = np.loadtxt(prototipo,delimiter=',')

###########################################################################
# Receptor

A partir de acá implemento lo que sería el receptor
##########################################################################

Divido los datos en canales

El conmutador del receptor rota en sentido contrario al del transmisor. Por eso se usa la función "flipud".

Reshape de los coeficientes del filtro en filtros polifásicos es decir, dividir en las $K$ polifases. 

Acá es donde armo los filtros polifásicos: $\bar{p}'_\rho(m) = h(mM-\rho),\quad \rho=0,1,...,M-1,\quad K=M$.
    
Todo esto, según pag. 361 de Crochiere y mis desarrollos en la pag. 186 de mi libro de notas.

In [ ]:
#Ep=np.zeros((K,int(proto.size/K))) 

In [ ]:
#for ro in range(K):
#    for m in range(int(proto.size/K)):
#        Ep[ro][m] = proto[int(M*m-ro)] 

In [ ]:
#for ro in range(1,K):
#    Ep[ro][0] = 0 

Para graficar las respuestas en el tiempo de cada uno de los filtros polifásicos

In [ ]:
#plt.figure()
#for i in range(K):
    #plt.subplots()
    #mfreqz(K*Ep[i])
#    h_freqz(K*Ep[i])
    #plt.savefig('plots/f{}.pdf'.format(i))
#    plt.tight_layout()
#plt.show()

Ahora analizo cuáles son las salidas respectivas de mis filtros para cada entrada FIR (filtros polifásicos)

FIR (polyphase filters)

In [ ]:
#rx_fir0_op_ideal = np.zeros((M,coe_datas.shape[1]),dtype=complex)
#rx_fir1_op_ideal = np.zeros((M,coe_datas.shape[1]),dtype=complex)
#rx_fir_op_ideal = np.zeros((K,coe_datas.shape[1]),dtype=complex)
#print('Run FIRs....');
#for n in range(M):
#   rx_fir0_op_ideal[n] = lfilter(filts0[n],1,coe_datas[n])
#for n in range(M):
#   rx_fir1_op_ideal[n] = lfilter(filts1[n],1,coe_datas[n])
#for n in range(K):
#    rx_fir_op_ideal[n] = lfilter(K*Ep[n],1,coe_datas[n])

Aquí multiplico por 2 en correspondencia con lo que hago en la línea 335 de gen_tonos_mkid.py. De esta forma tengo todas las simulaciones compatibilizadas.

Así coincide con la salida FIX19_20 que tengo en el FIR compiler. Esto hace que tenga a la salida del FIR los -24dB que corresponden con el diseño de los filtros y su ganancia (1/M) = (1/16)

In [ ]:
#rx_fir_op_ideal = 2*rx_fir_op_ideal

Ahora para compatibilizar la simulacion sin hacer el swap aun en vhdl lo hago acá

In [ ]:
#x0=np.reshape(x_fir0_sim,(int(x_fir0_sim.size/M),M)) 
#x1=np.reshape(x_fir1_sim,(int(x_fir1_sim.size/M),M))                                       
#x=np.array([])  
#for n in range(x0.shape[0]):
#       if n%2: 
#               x=np.append(x,np.concatenate([x0[n],x1[n]])) 
#       else: 
#               x=np.append(x,np.concatenate([x1[n],x0[n]])) 
#xc=np.reshape(x,(K,int(x.size/K)),order='F')  
viv_xc=np.reshape(viv_x_fir_op,(K,int(viv_x_fir_op.size/K)),order='F')
ideal_xc=np.reshape(x_ideal,(K,int(x_ideal.size/K)),order='F')

IFFT (mixers)

Gráfica de la salida de cada canal filtrado. Acá lo que hago es comparar la salida de Vivado con lo que me da la simulación de los filtros con Python.

In [ ]:
#Ahora grafico lo que obtengo a la salida del filtro polifásico del rx
#Grafico la salida de cada canal filtrado
fig,axs = plt.subplots(int(K/2),2,sharex=True, sharey=True, gridspec_kw={'wspace': 0.1})
fig.set_size_inches((10,8))
#fig.suptitle('Channelizer{}'.format(K),fontsize=18)

for n in range(0,int(K/2)):
    ax=axs[n][0]
    #ax.plot(rx_fir_op_ideal[n].real)
    ax.plot(2*K**2*ideal_xc[n].real)
    ax.plot(viv_xc[n].real,'r.',markevery=9)
    #ax.plot(2*K*(rx_fir_op_ideal[n][:viv_xc[n].size].real-viv_xc[n].real),'r.-',markevery=9)
    ax.set_ylabel(r'ch{} (V)'.format(n+1),fontsize=10)
    ax.set_xlim(0,1024)
    ax.grid()

    ax=axs[n][1]
    #ax.plot(rx_fir_op_ideal[n].imag)
    ax.plot(2*K**2*ideal_xc[n].imag)
    ax.plot(viv_xc[n].imag,'r.',markevery=9)
    ax.set_xlim(0,1024)
    ax.grid()
    axs[0][0].set_title('Real',fontsize=12)
axs[0][1].set_title('Imaginario',fontsize=12)
#axs[0][1].set_title('Imaginary',fontsize=12)
axs[int(K/2)-1][0].set_xlabel('n',fontsize=12)
axs[int(K/2)-1][1].set_xlabel('n',fontsize=12)
#fig.align_xlabels()
plt.tight_layout()
plt.savefig('plots/rx_chann{}_out_1_{}.pdf'.format(K,int(K/2)))
#plt.show()

In [ ]:
#Grafico la salida de cada canal
fig,axs = plt.subplots(int(K/2),2,sharex=True, sharey=True, gridspec_kw={'wspace': 0.1})
fig.set_size_inches((10,8))
#fig.suptitle('Channelizer{}'.format(K),fontsize=18)

for n in range(int(K/2),K):
    ax=axs[n-int(K/2)][0]
    #ax.plot(rx_fir_op_ideal[n].real)
    ax.plot(2*ideal_xc[n].real)
    ax.plot(viv_xc[n].real,'r.',markevery=9)
    ax.set_ylabel('ch{} (V)'.format(n+1),fontsize=10)
    ax.set_xlim(0,1024)
    ax.grid()

    ax=axs[n-int(K/2)][1]
    #ax.plot(rx_fir_op_ideal[n].imag)
    ax.plot(2*ideal_xc[n].imag)
    ax.plot(viv_xc[n].imag,'r.',markevery=9)
    ax.set_xlim(0,1024)
    ax.grid()

axs[0][0].set_title('Real',fontsize=12)
axs[0][1].set_title('Imaginario',fontsize=12)
#axs[0][1].set_title('Imaginary',fontsize=12)
axs[int(K/2)-1][0].set_xlabel('n',fontsize=12)
axs[int(K/2)-1][1].set_xlabel('n',fontsize=12)
#fig.align_xlabels()
plt.tight_layout()
plt.savefig('plots/rx_chann{}_out_{}_{}.pdf'.format(K,int(K/2+1),K))
plt.show()

Plot output spectrum. Estos son los datos ideales, solo para checkeo

In [ ]:
fig,ax=plt.subplots(figsize=(10,5))
plot_type = 'spectrum';
#plot_type = 'time domain(abs)';
#plot_type = 'time domain(real)';

print('Plot output channel {}....'.format(plot_type));
N=2**10
#ww=kaiser(data_len,8);
ww=kaiser(viv_xc[2].size,8);
#max_val = max(abs(fft(rx_fir_op_ideal[1]*ww)));
max_val = max(abs(fft(viv_xc[2]*ww,N)));
#f=fftfreq(data_len,d=1/Fs)
f=fftfreq(N,d=1/Fs)
for n in range(K):
    #chan_err = rx_fft_op_ideal(n,:) - rx_fft_op_core(n,:);
    if (plot_type =='spectrum'):
        #plt.plot(f,20*np.log10(abs(fft(rx_fir_op_ideal[n]*ww))/max_val),'b');
        ax.plot(fftshift(f),20*np.log10(abs(fftshift(fft(viv_xc[n]*ww,N)))/max_val),f'C{n}',label=f'ch{n}');
        #  plt.plot(20*np.log10(abs(fft(rx_fft_op_core[n]*ww))/max_val),'g');
        #  plt.plot(20*np.log10(abs(fft(chan_err*ww))/max_val),'r');
        ax.set_xlim(-Fs/2,Fs/2)
        ax.legend(loc='right')
        ax.grid()
        
    elif (plot_type =='time domain(real)'):
        #plt.plot(rx_fir_op_ideal[n].real,'b');
        #plt.plot(rx_fir_op_ideal[n].real,'b');
        ax.plot(viv_xc[3][:4000].real,'b');
        #  plt.plot(rx_fft_op_core[n].real,'g');
        #  plt.plot(abs(chan_err),'r');
    else:
        ax.plot(abs(rx_fir_op_ideal[n]),'b');
        #  plt.plot(abs(rx_fft_op_core[n]),'g');
        #  plt.plot(abs(chan_err),'r');
        ax.set_title('Receiver channel {:} {:}'.format(n,plot_type));
        #plt.legend('Ideal','Cores','Error');
plt.show()

Armamos las señales modulantes

In [ ]:
mod_sim  = viv_xc #np.reshape(x_sim,(K,int(len(x_sim)/K)),order='F') 
mod_ideal= np.reshape(x_ideal,(K,int(len(x_ideal)/K)),order='F')

In [ ]:
N=2**13         #longitud de la DFT
f = fftfreq(N,d=1/Fs);
ww_sim = kaiser(len(viv_xc[0]), beta=8)
ww_ideal = kaiser(len(ideal_xc[0]), beta=8)

In [ ]:
X_sim=[]
#PX_sim=[]
X_ideal=[]
#PX_ideal=[]
#max_val = max(abs(fft(rx_fft_op_ideal[2]*ww)))
for i in range(len(mod_sim)):
    #x_sim=fft(mod_sim[i][113:]*ww_sim,N);
    x_sim=fft(mod_sim[i]*ww_sim,N);
    #x_sim=x_sim[:int(N/2)];
#    p_sim=abs(x_sim)**2;
#    p_sim=10*np.log10(p_sim);
    X_sim.append(abs(x_sim))
#    PX_sim.append(p_sim)
for i in range(len(mod_ideal)):
    #x_ideal=fft(mod_ideal[i][113:]*ww_ideal,N);#tomo a partir de la muestra 113 porque son ceros antes de eso
    mx_ideal=fft(mod_ideal[i]*ww_ideal,N);#tomo a partir de la muestra 113 porque son ceros antes de eso
    #mx_ideal=mx_ideal[:int(N/2)];
#    p_ideal=abs(mx_ideal)**2;
#    p_ideal=10*np.log10(p_ideal);
    X_ideal.append(abs(mx_ideal)/max(abs(fft(mx_ideal[2]*ww_ideal,N))))
#    PX_ideal.append(p_ideal)

fig,ax = plt.subplots(2,1,figsize=(10,5),sharex=True)
#fig.set_size_inches((10,5))
#ahora grafico el espectro
for i in range(len(X_sim)):
    ax[0].plot(fftshift(f),20*np.log10(fftshift(X_sim[i])/max(abs(fft(x_sim[3]*ww_sim,N)))),label='CH{}'.format(i+1),lw=1)
for i in range(len(X_ideal)):
    ax[1].plot(fftshift(f),20*np.log10(fftshift(X_ideal[i])),label='CH{}'.format(i+1),lw=1)

#ax.autoscale()
ax[0].set_ylabel(r'Amplitud (dB)',fontsize=12)
ax[1].set_ylabel(r'Amplitud (dB)',fontsize=12)
ax[1].set_xlabel("Frecuencia (Hz)",fontsize=12)
#ax.set_ylim(-100,2)
#ax[0].set_xlim(left=0,right = Fs/2)
ax[1].set_xlim(left=0,right = Fs/2)
ax[0].set_ylim(top=40,bottom=-100)
ax[1].set_ylim(top=40,bottom=-100)
ax[0].grid()
ax[1].grid()
#ax.annotate('attenuated in\nstopband',fontsize=16,xy=(200,32),
#        xytext=(50,3),textcoords='offset points',
#        arrowprops=dict(arrowstyle='->',lw=3),)
ax[0].legend(loc='upper right',fontsize=5);
ax[1].legend(loc='upper right',fontsize=5);

# this is an inset axes over the main axes
a = plt.axes([.5, .58, .3, .3], alpha=0.2)
for i in range(0,len(X_sim)):
    #a.plot(f[:int(N/2)],X[i])
        a.plot(fftshift(f),20*np.log10(fftshift(X_sim[i])),label='CH{}'.format(i+1),lw=1)
        #a.plot(f[:int(N/2)],20*np.log10(X_ideal[i]),label='CH{}'.format(i+1),lw=1)
#for i in range(0,len(PX)):
#    a.plot(f[:int(N/2)],PX[i])

a.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
#a.set_xlabel('Charge [ADC.bin]',fontsize=14, fontname='monospace')
#a.set_ylabel('Counts',fontsize=14, fontname='monospace')
a.set_xlim(0,Fs/K)
a.set_ylim(-110,40)
#a.autoscale()
a.grid()

plt.savefig('plots/fir_fdm_out_chann{}.pdf'.format(K))
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(10,6))
#fig.set_size_inches((10,6))
#fig.suptitle('Input Spectrum',fontsize=18)

#NFFT=viv_x_fir_op.size
NFFT=1024
ww_viv = kaiser(viv_x_fir_op.size, beta=8)
ww_ideal = kaiser(x_ideal.size, beta=8)
X_viv=fft(viv_x_fir_op*ww_viv,NFFT)
X_ideal=fft(x_ideal*ww_ideal,NFFT)
freq=fftfreq(NFFT,d=1/Fs)
#for n in range(K):
ax.plot(fftshift(freq),20*np.log10(fftshift(abs(X_ideal/max(abs(X_ideal))))),'r',label='Python')
ax.plot(fftshift(freq),20*np.log10(fftshift(abs(X_viv/max(abs(X_viv))))),label='Vivado')
#ax.plot(fftshift(freq),20*np.log10(fftshift(abs(X_viv))),label='Vivado')
#ax.plot(fftshift(freq),20*np.log10(fftshift(abs(X_ideal))),label='Python')

for i in range(int(K/2)):
    ax.vlines((Fs/K)*i,-120,0,linestyle='dotted',color='red')
for i in reversed(range(int(K/2))):
    ax.vlines(-(Fs/K)*i,-120,0,linestyle='dotted',color='red')

ax.set_xlim(-Fs/2,Fs/2)
ax.set_ylim(-55,2)
ax.grid(ls='dotted')
ax.legend(loc=0,fontsize=12)
ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
ax.set_ylabel(r'Amplitud (dB)',fontsize=14)
ax.set_xlabel('Frecuencia (Hz)',fontsize=14)
plt.tight_layout()
#plt.savefig('plots/fir_out_viv_py.pdf')
plt.show()

In [ ]:
#armamos la señal total
#x_tot = x_sim1+x_sim2+x_sim3+x_sim4+x_sim5+x_sim6+x_sim7+x_sim8+x_sim9+x_sim10+x_sim11+x_sim12+x_sim13+x_sim14+x_sim15+x_sim16;
x_tot_sim = mod_sim.sum(axis=0)
x_tot_ideal = mod_ideal.sum(axis=0)

#graficamos una de las componentes
N=2**13         #longitud de la DFT
f = fftfreq(N,d=1/Fs);

#ww=kaiser(length(x_sim1),16);
ww_sim = kaiser(len(mod_sim[0][113:]), beta=8)
ww_ideal = kaiser(len(mod_ideal[0][113:]), beta=8)

X_sim=[]
PX_sim=[]
X_ideal=[]
PX_ideal=[]
for i in range(len(mod_sim)):
    x_sim=fft(mod_sim[i][113:]*ww_sim,N)/max(abs(fft(mod_sim[2][113:]*ww_sim,N)));#tomo a partir de la muestra 113 porque son ceros antes de eso
    #x_sim=x_sim[:int(N/2)];
    p_sim=abs(x_sim)**2;
    p_sim=10*np.log10(p_sim);
    X_sim.append(abs(x_sim))
    PX_sim.append(p_sim)
for i in range(len(mod_ideal)):
    x_ideal=fft(mod_ideal[i][113:]*ww_ideal,N)/max(abs(fft(mod_ideal[2][113:]*ww_ideal,N)));#tomo a partir de la muestra 113 porque son ceros antes de eso
    #x_ideal=x_ideal[:int(N/2)];
    p_ideal=abs(x_ideal)**2;
    p_ideal=10*np.log10(p_ideal);
    X_ideal.append(abs(x_ideal))
    PX_ideal.append(p_ideal)

fig,ax = plt.subplots(2,1,sharex=True,figsize=(10,6))
#fig.set_size_inches((10,5))
#ahora grafico el espectro
for i in range(len(X_sim)):
        #ax[0].plot(f[:int(N/2)],20*np.log10(X_sim[i]),label='CH{}'.format(i+1),lw=1)
        ax[0].plot(fftshift(f),20*np.log10(fftshift(X_sim[i])),label='CH{}'.format(i+1),lw=1)
for i in range(len(X_ideal)):
        #ax[1].plot(f[:int(N/2)],20*np.log10(X_ideal[i]),label='CH{}'.format(i+1),lw=1)
        ax[1].plot(fftshift(f),20*np.log10(fftshift(X_ideal[i])),label='CH{}'.format(i+1),lw=1)

#fig,ax=plt.subplots()
##ahora grafico la densidad espectral de potencia
#for i in range(len(PX)):
#    plt.plot(f[:int(N/2)],PX[i],label='CH{}'.format(i+1))

#ax.autoscale()
ax[0].set_ylabel(r'Amplitud (dB)',fontsize=12)
ax[1].set_ylabel(r'Amplitud (dB)',fontsize=12)
ax[1].set_xlabel("Frequencia (Hz)",fontsize=12)
#ax.set_ylim(-100,2)
ax[0].set_xlim(left=0,right = Fs/2)
ax[1].set_xlim(left=0,right = Fs/2)
ax[0].set_ylim(top=2,bottom=-85)
ax[1].set_ylim(top=2,bottom=-75)
ax[0].grid()
ax[1].grid()
#ax.annotate('attenuated in\nstopband',fontsize=16,xy=(200,32),
#        xytext=(50,3),textcoords='offset points',
#        arrowprops=dict(arrowstyle='->',lw=3),)
ax[0].legend(loc='upper right',fontsize=7);
ax[1].legend(loc='upper right',fontsize=7);
# this is an inset axes over the main axes
a = plt.axes([.5, .64, .3, .3], alpha=0.02)
for i in range(0,len(X_sim)):
    #a.plot(f[:int(N/2)],X[i])
        a.plot(fftshift(f),20*np.log10(fftshift(X_sim[i])),label='CH{}'.format(i+1),lw=1)
        #a.plot(f[:int(N/2)],20*np.log10(X_ideal[i]),label='CH{}'.format(i+1),lw=1)
#for i in range(0,len(PX)):
#    a.plot(f[:int(N/2)],PX[i])

a.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
#a.set_xlabel('Charge [ADC.bin]',fontsize=14, fontname='monospace')
#a.set_ylabel('Counts',fontsize=14, fontname='monospace')
a.set_xlim(0,Fs/K)
a.set_ylim(-70,2)
#a.autoscale()
a.grid()
plt.tight_layout()

plt.savefig('plots/fir_fdm_out_chann{}.pdf'.format(K))
plt.show()
